# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

weatherpy_csv = "output_data_file"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacationpy_df = pd.read_csv(weatherpy_csv)
vacationpy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hervey bay,-25.2986,152.8535,73.00,63,52,8.01,AU,1619495977
1,belmonte,-15.8631,-38.8828,75.20,78,20,5.75,BR,1619495978
2,punta arenas,-53.1500,-70.9167,44.60,93,90,5.75,CL,1619495978
3,geraldton,-28.7667,114.6000,87.80,20,20,3.44,AU,1619495979
4,oranjemund,-28.5500,16.4333,59.04,86,79,5.12,NaN,1619495979


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
locations = vacationpy_df[["Lat", "Lng"]]
hum = vacationpy_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
### Create new DataFrame fitting weather criteria
# * Narrow down the cities to fit weather conditions.
# * Drop any rows will null values.

In [6]:
vacationpy_weather_crit = vacationpy_df[vacationpy_df['Max Temp'] < 80] 
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Max Temp'] > 70]
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Wind Speed'] < 10]
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Cloudiness'] == 0]
vacationpy_weather_crit

<ipython-input-6-1ea31caf26ac>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Max Temp'] > 70]
<ipython-input-6-1ea31caf26ac>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Wind Speed'] < 10]
<ipython-input-6-1ea31caf26ac>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Cloudiness'] == 0]


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
195,lenger,42.1830,69.8839,73.4,38,0,2.24,KZ,1619496084
250,casa nova,-9.4167,-41.1333,78.8,65,0,9.22,BR,1619496107
302,meybod,32.2501,54.0166,71.6,37,0,2.30,IR,1619496128
421,conceicao do araguaia,-8.2578,-49.2647,77.0,94,0,3.44,BR,1619496177
423,riyadh,24.6877,46.7219,77.0,22,0,8.68,SA,1619496007
425,dubbo,-32.2500,148.6167,75.2,29,0,2.30,AU,1619496178
570,silvania,-16.6589,-48.6081,73.4,73,0,4.61,BR,1619496237


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
vacationpy_weather_crit['Hotel Name'] = ''
vacationpy_weather_crit = vacationpy_weather_crit.reset_index(drop=True)
vacationpy_weather_crit.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,lenger,42.1830,69.8839,73.4,38,0,2.24,KZ,1619496084,
1,casa nova,-9.4167,-41.1333,78.8,65,0,9.22,BR,1619496107,
2,meybod,32.2501,54.0166,71.6,37,0,2.30,IR,1619496128,
3,conceicao do araguaia,-8.2578,-49.2647,77.0,94,0,3.44,BR,1619496177,
4,riyadh,24.6877,46.7219,77.0,22,0,8.68,SA,1619496007,


In [9]:
vacationpy_weather_crit[['Lat','Lng']]

,Lat,Lng
0,42.1830,69.8839
1,-9.4167,-41.1333
2,32.2501,54.0166
3,-8.2578,-49.2647
4,24.6877,46.7219
5,-32.2500,148.6167
6,-16.6589,-48.6081


In [60]:
hotel_names = []
cities = []
countries = []
lats = []
lngs = []

for vacay in range(len(vacationpy_weather_crit)):
    target_coordinates_lat = vacationpy_weather_crit.loc[vacay]['Lat']
    target_coordinates_lng = vacationpy_weather_crit.loc[vacay]['Lng']
    target_coordinates_city = vacationpy_weather_crit.loc[vacay]['City']
    target_coordinates_country = vacationpy_weather_crit.loc[vacay]['Country']


    # geocoordinates
    target_type = "lodging"
    target_radius = 5000

    # set up a parameters dictionary
    params = {
        "location": f"{target_coordinates_lat}, {target_coordinates_lng}",
        "radius": target_radius,
        "type": target_type,
#         "City": target_coordinates_city,
#         "Country": target_coordinates_country,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    time.sleep(2)
    cities.append(target_coordinates_city)
    countries.append(target_coordinates_country)
    lats.append(target_coordinates_lat)
    lngs.append(target_coordinates_lng)

    if(response.status_code==200):
        hotel_json = response.json()
    
        first_hotel = hotel_json['results'][0]
        first_hotel_name = first_hotel['name']
#         print(first_hotel_name)
        hotel_names.append(first_hotel_name)
    else:
        hotel_names.append('')
        print(f'Error: {params}')

hotel_df = pd.DataFrame({
    'Hotel Name': hotel_names,
    'City': cities,
    'Country': countries,
    'Lat': lats,
    'Lng': lngs,
                        })

In [61]:
for index, row in hotel_df.iterrows():
    print(row)

Hotel Name    Gostevoy Dom 1
City                  lenger
Country                   KZ
Lat                   42.183
Lng                  69.8839
Name: 0, dtype: object
Hotel Name    Pousada e Restaurante Dona Xinha
City                                 casa nova
Country                                     BR
Lat                                    -9.4167
Lng                                   -41.1333
Name: 1, dtype: object
Hotel Name    Traditional Hotel Restaurant Haji Malek
City                                           meybod
Country                                            IR
Lat                                           32.2501
Lng                                           54.0166
Name: 2, dtype: object
Hotel Name    Tarumã Tropical Hotel
City          conceicao do araguaia
Country                          BR
Lat                         -8.2578
Lng                        -49.2647
Name: 3, dtype: object
Hotel Name    InterContinental Riyadh, an IHG Hotel
City                      

In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# Add marker layer ontop of heat map
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,map_type='SATELLITE')
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…